# 1. Import required modules

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import data
import skimage
from skimage.filters.thresholding import threshold_li,threshold_local,threshold_otsu
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, remove_small_objects, ball
from skimage.morphology import disk
from scipy import ndimage as ndi
import sys,os, glob

#import skimage.filters.median

#import skimage.segmentation as seg
#import skimage.filters as filters
#import skimage.draw as draw
#import skimage.color as color

#from skimage.filters.thresholding import _cross_entropy
#from skimage.morphology import black_tophat, skeletonize, convex_hull_image


#pip install nd2reader_required for nd2 file reading
from nd2reader import ND2Reader

import pandas as pd

c:\programdata\anaconda3\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


# 2. Define common parameters for measurement loading here:

In [4]:
# The labeled histone mark in the excel

h2ax_key = 'H3K27me3'


# the actual histone mark analyzed  (to correct the mis-labeling in the raw measurements)
actual_h2ax_key = 'H3T3'

## Control exps

### Batch 1

In [5]:
# load experiment exp1

analysis_fd = r'D:\Analyzed_CellBio\Stam\Revision_exps\220307IF_220214exp_noco45h_Mdc1_H3T3_H3S10\cov1_H3T3\\analysis_subset_controlled'

exp1_df = pd.read_excel(analysis_fd + os.sep + 'measurement_for_all_fov_background_subtracted_2_100_crop.xlsx')

oldnames = [_c for _c in exp1_df.columns if h2ax_key in _c]
newnames = [_c.replace(h2ax_key,actual_h2ax_key) for _c in oldnames]
rename_dict = {_o:_n for _o, _n in zip(oldnames,newnames)}
exp1_df = exp1_df.rename(columns=rename_dict)


exp1_df['exp_annotation']='220307IF_220214exp'

In [6]:
# load experiment exp2

analysis_fd = r'D:\Analyzed_CellBio\Stam\Revision_exps\2203087IF_220216exp_noco45h_Mdc1_H3T3_H3S10\cov1_H3T3\analysis_subset_controlled'

exp2_df = pd.read_excel(analysis_fd + os.sep + 'measurement_for_all_fov_background_subtracted_2_100_crop.xlsx')

oldnames = [_c for _c in exp2_df.columns if h2ax_key in _c]
newnames = [_c.replace(h2ax_key,actual_h2ax_key) for _c in oldnames]
rename_dict = {_o:_n for _o, _n in zip(oldnames,newnames)}
exp2_df = exp2_df.rename(columns=rename_dict)


exp2_df['exp_annotation']='2203087IF_220216exp'

### Batch 2

In [7]:
# load experiment exp3

analysis_fd = r'D:\Analyzed_CellBio\Stam\Revision_exps_p2\220613IF_newCovs_H3T3_45h\220307exp_H3T3_newCov_45h\analysis_subset_controlled'

exp3_df = pd.read_excel(analysis_fd + os.sep + 'measurement_for_all_fov_background_subtracted_2_100_crop.xlsx')

exp3_df['exp_annotation']='220613IF_220307exp'

In [8]:
ctrl_combined_df = pd.concat([exp1_df,exp2_df,exp3_df])

len(ctrl_combined_df)


#measurement_for_all_fov_df = ctrl_combined_df

268

In [9]:
ctrl_combined_df.columns

Index(['Unnamed: 0', 'exp_name', 'position_id', 'cell_id', 'cell_xyz_coord',
       'area_incorporated_chr', 'area_h2ax_within_incorporated_chr',
       'area_less_h2ax_within_incorporated_chr', 'area_control_chr',
       'area_nucleo-excluded_control_chr', 'area_nucleolus',
       'area_subset_control_ctrl', 'ave_intensity_incorporated_chr_dna',
       'ave_intensity_control_chr_dna',
       'ave_intensity_nucleo-excluded_control_chr_dna',
       'ave_intensity_nucleolus_dna', 'ave_intensity_subset_control_chr_dna',
       'ave_intensity_h2ax_pos_incorporated_chr_dna',
       'ave_intensity_h2ax_less_incorporated_chr_dna',
       'ave_intensity_incorporated_chr_Mdc1', 'ave_intensity_control_chr_Mdc1',
       'ave_intensity_nucleo-excluded_control_chr_Mdc1',
       'ave_intensity_nucleolus_Mdc1', 'ave_intensity_subset_control_chr_Mdc1',
       'ave_intensity_h2ax_pos_incorporated_chr_Mdc1',
       'ave_intensity_h2ax_less_incorporated_chr_Mdc1',
       'ave_intensity_incorporated_chr_H

# 3. Post-hoc analysis examples

In [10]:
#len(measurement_for_all_fov_df)


analysis_save_path = r'D:\Analyzed_CellBio\Stam\Revision_exps_p2\H3S10_and_H3T3_analysis'

analysis_savename = 'mdc-568_h3t3_pol2_analyzed.xlsx'

if not os.path.exists(analysis_save_path):
    os.mkdir(analysis_save_path)

In [11]:
# Process each datafram

out_df_dict = {}
exp_name_list = ["Control"]

for _df_index, _df in enumerate([ctrl_combined_df]):
    print(f'analyzing experiment group {_df_index}.')

    # remove the bad cell that has no Pol2 signal
    analysis_df = _df[_df['ave_intensity_control_chr_Pol2S5']>0].copy()
    print (f'-- There are {len(analysis_df)} data-points after filtering.')

    # for total incorporated chr
    analysis_df['normalized_inc_Pol2S5']=analysis_df['ave_intensity_incorporated_chr_Pol2S5']/analysis_df['ave_intensity_nucleo-excluded_control_chr_Pol2S5']
    print(analysis_df['normalized_inc_Pol2S5'].mean())

    # for subset control
    analysis_df['normalized_subset_control_Pol2S5']=analysis_df['ave_intensity_subset_control_chr_Pol2S5']/analysis_df['ave_intensity_nucleo-excluded_control_chr_Pol2S5']
    print(analysis_df['normalized_subset_control_Pol2S5'].mean())

    # get the h2ax-positive area vs h2ax-less (positive) area for the same cell
    analysis_df[f'normalized_inc_{actual_h2ax_key}'] = analysis_df[f'ave_intensity_incorporated_chr_{actual_h2ax_key}']/analysis_df[f'ave_intensity_nucleo-excluded_control_chr_{actual_h2ax_key}']
    print(analysis_df[f'normalized_inc_{actual_h2ax_key}'].mean())

    # for subset control
    analysis_df[f'normalized_subset_control_{actual_h2ax_key}']=analysis_df[f'ave_intensity_subset_control_chr_{actual_h2ax_key}']/analysis_df[f'ave_intensity_nucleo-excluded_control_chr_{actual_h2ax_key}']
    print(analysis_df[f'normalized_subset_control_{actual_h2ax_key}'].mean())

    # get the h2ax-positive area vs h2ax-less (positive) area for the same cell
    analysis_df['normalized_inc_Mdc1'] = analysis_df['ave_intensity_incorporated_chr_Mdc1']/analysis_df['ave_intensity_nucleo-excluded_control_chr_Mdc1']
    print(analysis_df['normalized_inc_Mdc1'].mean())

    # for subset control
    analysis_df['normalized_subset_control_Mdc1']=analysis_df['ave_intensity_subset_control_chr_Mdc1']/analysis_df['ave_intensity_nucleo-excluded_control_chr_Mdc1']
    print(analysis_df['normalized_subset_control_Mdc1'].mean())

    # get the h2ax-positive area vs h2ax-less (positive) area for the same cell
    analysis_df['normalized_inc_dna'] = analysis_df['ave_intensity_incorporated_chr_dna']/analysis_df['ave_intensity_nucleo-excluded_control_chr_dna']
    print(analysis_df['normalized_inc_dna'].mean())

    # for subset control
    analysis_df['normalized_subset_control_dna']=analysis_df['ave_intensity_subset_control_chr_dna']/analysis_df['ave_intensity_nucleo-excluded_control_chr_dna']
    print(analysis_df['normalized_subset_control_dna'].mean())
    
    # reorder some column
    col = analysis_df.pop("exp_annotation")
    analysis_df.insert(0, "exp_annotation", col)
    
    out_df_key = exp_name_list[_df_index]
    out_df_dict[out_df_key]=analysis_df
    

analyzing experiment group 0.
-- There are 268 data-points after filtering.
0.8318845161807087
1.0235384799168075
1.1048937120185875
1.02616096566035
4.174450928177603
1.0014652343186812
1.0613965132023457
1.0010189270048933


In [12]:
#analysis_savename = 'rpe1_h3k27_pol2_analyzed.xlsx'
for _exp_key, _exp_df in out_df_dict.items():
    exp_analysis_savename = _exp_key+'_'+analysis_savename
    _exp_df.to_excel(os.path.join(analysis_save_path,exp_analysis_savename), index=False)

## Size filtering with 300 pixels

In [13]:
size_th =300
analysis_filtered_savename = analysis_savename.split('.xlsx')[0]+ f'_size{size_th}' + '.xlsx'


for _exp_key, _exp_df in out_df_dict.items():
    
    _exp_df= _exp_df[_exp_df['area_incorporated_chr']>size_th].copy()
    print (f'-- There are {len(_exp_df)} data-points after filtering.')
    exp_analysis_savename = _exp_key+'_'+analysis_filtered_savename
    _exp_df.to_excel(os.path.join(analysis_save_path,exp_analysis_savename), index=False)

-- There are 143 data-points after filtering.


## Postion filtering

In [14]:
def filter_cell_by_coord (xyz_coord_str, bound1=2048, bound2=0, dist_tp_bound = 600):
    
    xy_coord_str = xyz_coord_str.split('],')[0].split('[[')[-1]
    # note here xy is symetric; otherwise need to check which is x and which is y
    x=int(xy_coord_str.split(',')[0])
    y=int(xy_coord_str.split(',')[1])
    
    if abs(x-bound1)>=dist_tp_bound and abs(y-bound1)>=dist_tp_bound:
        if abs(x-bound2)>=dist_tp_bound and abs(y-bound2)>=dist_tp_bound:
            result= 1
        else:
            result= 0
    else:
        result= 0

    return result
        
    
analysis_filtered_savename = analysis_savename.split('.xlsx')[0]+ f'_size{size_th}_pos_filtered' + '.xlsx'


for _exp_key, _exp_df in out_df_dict.items():    
    
    # size filtering
    _exp_df= _exp_df[_exp_df['area_incorporated_chr']>size_th].copy()
    print (f'-- There are {len(_exp_df)} data-points after size filtering.')
    # postion filtering
    _exp_df['pos_kept']=_exp_df['cell_xyz_coord'].map(lambda x: filter_cell_by_coord(x))
    _exp_df=_exp_df[_exp_df['pos_kept']==1].copy()
    print (f'-- There are {len(_exp_df)} data-points after position filtering.')
    exp_analysis_savename = _exp_key+'_'+analysis_filtered_savename
    _exp_df.to_excel(os.path.join(analysis_save_path,exp_analysis_savename), index=False)


-- There are 143 data-points after size filtering.
-- There are 124 data-points after position filtering.


In [ ]:
# for dna

# 
g1 = out_df_dict['Control']['normalized_inc_dna']
g2 = out_df_dict['Control']['normalized_subset_control_dna']
#g3 = out_df_dict['HDACi']['normalized_inc_dna']
#g4 = out_df_dict['HDACi']['normalized_subset_control_dna']

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
# for mdc1

g1 = out_df_dict['Control']['normalized_inc_Mdc1']
g2 = out_df_dict['Control']['normalized_subset_control_Mdc1']
#g3 = out_df_dict['HDACi']['normalized_inc_Mdc1']
#g4 = out_df_dict['HDACi']['normalized_subset_control_Mdc1']

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
# For Pol2S5

g1 = out_df_dict['Control']['normalized_inc_Pol2S5']
g2 = out_df_dict['Control']['normalized_subset_control_Pol2S5']
#g3 = out_df_dict['HDACi']['normalized_inc_Pol2S5']
#g4 = out_df_dict['HDACi']['normalized_subset_control_Pol2S5']

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
# For Pol2S5 norm to dna

g1 = (out_df_dict['Control']['normalized_inc_Pol2S5'])/(out_df_dict['Control']['normalized_inc_dna'])
g2 = (out_df_dict['Control']['normalized_subset_control_Pol2S5'])/(out_df_dict['Control']['normalized_subset_control_dna'])
#g3 = out_df_dict['HDACi']['normalized_inc_Pol2S5']/(out_df_dict['HDACi']['normalized_inc_dna'])
#g4 = out_df_dict['HDACi']['normalized_subset_control_Pol2S5']/(out_df_dict['HDACi']['normalized_subset_control_dna'])

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
# For H2ax-key relevant Ab


g1 = (out_df_dict['Control'][f'normalized_inc_{actual_h2ax_key}'])
g2 = (out_df_dict['Control'][f'normalized_subset_control_{actual_h2ax_key}'])
#g3 = out_df_dict['HDACi'][f'normalized_inc_{actual_h2ax_key}']
#g4 = out_df_dict['HDACi'][f'normalized_subset_control_{actual_h2ax_key}']

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
# For H2ax-key relevant Ab


g1 = (out_df_dict['Control'][f'normalized_inc_{actual_h2ax_key}'])/(out_df_dict['Control']['normalized_inc_dna'])
g2 = (out_df_dict['Control'][f'normalized_subset_control_{actual_h2ax_key}'])/(out_df_dict['Control']['normalized_subset_control_dna'])
#g3 = out_df_dict['HDACi'][f'normalized_inc_{actual_h2ax_key}']/(out_df_dict['HDACi']['normalized_inc_dna'])
#g4 = out_df_dict['HDACi'][f'normalized_subset_control_{actual_h2ax_key}']/(out_df_dict['HDACi']['normalized_subset_control_dna'])

#sp_low = min(len(g1),len(g2),len(g3))
#sp_low = min(len(g1),len(g2), len(g3))

#g1 = np.random.choice(g1, sp_low)
#g2 = np.random.choice(g2, sp_low)
#g3 = np.random.choice(g3, sp_low)
#g4 = np.random.choice(g4, sp_low)
args = (g1,g2)
boxplot_pol2= plt.boxplot([*args], meanline = True)


vals, xs = [], [] 

for i, subdf in enumerate([*args]):
    #names.append(name)
    vals.append(subdf.tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

for x, val in zip(xs, vals):
    plt.scatter(x, val,  s = 1, alpha=0.3)

from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
#p4 = kruskal(g1,g2,g3,g4)

U1, p1 = mannwhitneyu(g1,g2)

print(f"MW comparision: {p1}")



#print(f"three samples comparision: {p4}")

In [ ]:
from scipy import stats
stats.spearmanr(out_df_dict['Control']['normalized_inc_Pol2S5'], out_df_dict['Control'][f'normalized_inc_{actual_h2ax_key}'])

In [ ]:
plt.scatter(x= out_df_dict['Control']['normalized_inc_Pol2S5'], y =out_df_dict['Control'][f'normalized_inc_{actual_h2ax_key}'])


In [ ]:
plt.scatter(x= out_df_dict['HDACi']['normalized_inc_Pol2S5'], y =out_df_dict['HDACi'][f'normalized_inc_{actual_h2ax_key}'])
